# GHCNd station data

These are data from Global Historical Climatology Network daily (GHCNd)

> https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily
>
> The Global Historical Climatology Network daily (GHCNd) is an integrated database of daily climate summaries from land surface stations across the globe. GHCNd is made up of daily climate records from numerous sources that have been integrated and subjected to a common suite of quality assurance reviews.
>
> GHCNd contains records from more than 100,000 stations in 180 countries and territories. NCEI provides numerous daily variables, including maximum and minimum temperature, total daily precipitation, snowfall, and snow depth. About half the stations only report precipitation. Both record length and period of record vary by station and cover intervals ranging from less than a year to more than 175 years.


In [ ]:
import time
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
import yaml
import sys

ghcnd_stations_url = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/doc/ghcnd-stations.txt'
ghcnd_stations_url = '/home/javi/Research/cordex-fps-urb-rcc/git/cordex-fps-urb-rcc/ghcnd-stations.txt'
ghcnd_stations_column_names = ['code', 'lat', 'lon', 'elev', 'name', 'net', 'numcode']
ghcnd_stations_column_widths = [   11,     9,    10,      7,     34,     4,       10 ]

In [ ]:
df = pd.read_fwf(ghcnd_stations_url, header = 0,
  widths = ghcnd_stations_column_widths,
  names = ghcnd_stations_column_names
)
ghcnd_stations = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'EPSG:4326')

In [ ]:
p = ghcnd_stations.plot(figsize=(20,10), markersize=2)

## Read city databases

Read in OECC city info

In [ ]:
df = pd.read_csv('city_info_out.csv', comment='#',
  dtype = dict(domain = 'category', ktype = 'category')
)
city_info = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs = 'EPSG:4326')

and the names of the currently selected cities

In [ ]:
with open('selected_cities.yaml') as fp:
  selected = yaml.load(fp, Loader=yaml.FullLoader)['cordex_core_analysis']

## Plot nearby stations (functions)

By default, we select those within half a degree around the city center

In [ ]:
def nearby_stations(city, maxdis = 0.5):
  citydf = city_info.query(f'city == "{city}"').squeeze()
  nearby_stations = ghcnd_stations[ghcnd_stations.distance(citydf.geometry) < maxdis].to_crs(epsg=3857)
  return(nearby_stations)

def plot_nearby_stations(city, nearby_stations, maxdis = 0.5, highlight=pd.DataFrame()):
  citydf = city_info.query(f'city == "{city}"').squeeze()
  img, ext = cx.bounds2img(
    citydf.lon - maxdis, citydf.lat - maxdis, citydf.lon + maxdis, citydf.lat + maxdis, ll=True,
    # source=cx.providers.Stamen.Toner
    # source=cx.providers.OpenStreetMap.Mapnik
  )
  ax = nearby_stations.plot(figsize=(15,15), color = 'red')
  for x, y, label in zip(nearby_stations.geometry.x, nearby_stations.geometry.y, nearby_stations.name):
    ax.annotate(label, xy=(x, y), xytext=(10, -10), textcoords='offset points', color = 'red')
  if not highlight.empty:
    highlight.plot(ax=ax, color='blue', edgecolor='white', zorder=100)
  for x, y, label in zip(highlight.geometry.x, highlight.geometry.y, highlight.name):
    ax.annotate(label, xy=(x, y), xytext=(10, -10), textcoords='offset points', color = 'blue', fontsize=20)
  ax.imshow(img, extent=ext)
  plt.title(city)
  plt.savefig('results/pictures/Nearby_stations_cycle_' + city + '.png', facecolor='white')

  #cx.add_basemap(ax, reset_extent = False)

def get_ghcnd_df(code):
    baseurl = 'http://meteo.unican.es/work/chus/ghcnd/data'
    try:
      rval = pd.read_csv(f'{baseurl}/{code[0]}/{code}.csv.gz', compression='gzip', index_col='DATE', parse_dates=True)
    except:
      print(f'Problem downloading {code}')
      rval = pd.DataFrame()
    return(rval)

def available_vars(station):
  return(set(station.columns).intersection({'TMIN'}))
  #return(set(station.columns).intersection({'PRCP', 'TAVG', 'TMAX', 'TMIN', 'SNWD'}))

def plot_station_timeseries(city, stations, var = 'PRCP', v_records = 0.5, idate='1979-01-01', fdate='2014-12-31'):
  period = slice(idate, fdate)
  ndays = (pd.to_datetime(fdate)-pd.to_datetime(idate)).days
  valid_codes = []
  for stn_code in stations.code:
    stn_data = get_ghcnd_df(stn_code)
    if stn_data.empty:
      continue
    availvars = available_vars(stn_data)
    print(f'{stn_data.NAME[0]} {availvars}', end=' ')
    if var in availvars:
      valid_records = stn_data[var].loc[period].notna().sum()/ndays
      print(f'  {var} has {100*valid_records:.1f}% valid records in {idate} to {fdate}')
      if valid_records > v_records:
        plt.figure()
        stn_data[var].loc[period].plot(
          figsize=(14,3),
          xlim=(idate,fdate),
          title = f'{city} -- {stn_data.NAME[0]} ({100*valid_records:.0f}%)'
        )
        valid_codes.append(stn_code)
    else:
      print(f'  No {var}')
  return(stations[stations.code.isin(valid_codes)])

In [ ]:
def plot_city(city_name, maxdis = 0.5, v_records = 0.3):
  nearstat = nearby_stations(city_name, maxdis = maxdis)
  if not nearstat.empty:
    valid = plot_station_timeseries(city_name, nearstat, 'TMIN', v_records)
    plot_nearby_stations(city_name, nearstat, maxdis= maxdis, highlight=valid)
  else:
    print('No nearby stations in GHCNd')

In [ ]:
import warnings
warnings.filterwarnings('ignore') # Hide UserWarnings alerting on the approximate distance in geographic CRS

# Selected cities

For each city, surrounding GHCNd stations with valid values above 50% are shown as time series. The city map displaus all surrounding stations (in red) along with those meeting the valid data criterion (in blue).

Next cell contents generated by:

```python
for city in selected:
  print(f'## {city}\nplot_city("{city}")')
```

```python
print(' · '.join([f'[{city}](#{city.replace(" ","-")})' for city in selected]))
```

[Cairo](#Cairo) · [Lagos](#Lagos) · [Johannesburg](#Johannesburg) · [Luanda](#Luanda) · [Khartoum](#Khartoum) · [Sydney](#Sydney) · [Melbourne](#Melbourne) · [Mexico City](#Mexico-City) · [Moscow](#Moscow) · [Tashkent](#Tashkent) · [Tehran](#Tehran) · [Dhaka](#Dhaka) · [Beijing](#Beijing) · [Tokyo](#Tokyo) · [Chengdu](#Chengdu) · [Seoul](#Seoul) · [Shanghai](#Shanghai) · [Paris](#Paris) · [London](#London) · [Istanbul](#Istanbul) · [Berlin](#Berlin) · [Helsinki](#Helsinki) · [New York](#New-York) · [Chicago](#Chicago) · [Los Angeles](#Los-Angeles) · [Montreal](#Montreal) · [Toronto](#Toronto) · [Buenos Aires](#Buenos-Aires) · [Lima](#Lima) · [São Paulo](#São-Paulo) · [Santiago](#Santiago) · [Jakarta](#Jakarta) · [Quezon City [Manila]](#Quezon-City-[Manila]) · [Singapore](#Singapore) · [Mumbai](#Mumbai) · [Riyadh](#Riyadh) · [Delhi [New Delhi]](#Delhi-[New-Delhi]) · [Baghdad](#Baghdad)

## Sydney

In [ ]:
plot_city("London", maxdis = 0.5, v_records = 0.5)

## Lagos

In [ ]:
plot_city("Lagos")

## Johannesburg

In [ ]:
plot_city("Johannesburg")

## Luanda

In [ ]:
plot_city("Luanda")

## Khartoum

In [ ]:
plot_city("Khartoum")

## Sydney

In [ ]:
plot_city("Sydney")

## Melbourne

In [ ]:
plot_city("Melbourne")

## Mexico City

In [ ]:
plot_city("Mexico City")

## Moscow

In [ ]:
plot_city("Moscow")

## Tashkent

In [ ]:
plot_city("Tashkent")

## Tehran

In [ ]:
plot_city("Tehran")

## Dhaka

In [ ]:
plot_city("Dhaka")

## Beijing

In [ ]:
plot_city("Beijing", maxdis = 2)

## Tokyo

In [ ]:
plot_city("Tokyo")

## Chengdu

In [ ]:
plot_city("Chengdu")

## Seoul

In [ ]:
plot_city("Seoul")

## Shanghai

In [ ]:
plot_city("Shanghai")

## Paris

In [ ]:
plot_city("Paris")

## London

In [ ]:
plot_city("London")

## Istanbul

In [ ]:
plot_city("Istanbul")

## Berlin

In [ ]:
plot_city("Berlin")

## Helsinki

In [ ]:
plot_city("Helsinki")

## New York

In [ ]:
plot_city("New York")

## Chicago

In [ ]:
plot_city("Chicago")

## Los Angeles

In [ ]:
plot_city("Los Angeles")

## Montreal

In [ ]:
plot_city("Montreal")

## Toronto

In [ ]:
plot_city("Toronto")

## Buenos Aires

In [ ]:
plot_city("Buenos Aires")

## Lima

In [ ]:
plot_city("Lima")

## São Paulo

In [ ]:
plot_city("São Paulo")

## Santiago

In [ ]:
plot_city("Santiago")

## Jakarta

In [ ]:
plot_city("Jakarta")

## Quezon City [Manila]

In [ ]:
plot_city("Quezon City [Manila]")

## Singapore

In [ ]:
plot_city("Singapore")

## Mumbai

In [ ]:
plot_city("Mumbai")

## Riyadh

In [ ]:
plot_city("Riyadh")

## Delhi [New Delhi]

In [ ]:
plot_city("Delhi [New Delhi]")

## Baghdad

In [ ]:
plot_city("Baghdad")